In [1]:
import functools
import json
import math
import pandas as pd
import pennylane as qml
import pennylane.numpy as np
import scipy

In [2]:
dev = qml.device("default.qubit", wires=2)

@qml.qnode(dev)
def circuit(params):
    #f(Theta_bar)
    qml.RY(params[0], 0)
    qml.RX(params[1], 1)
    return qml.expval(qml.PauliZ(0) + qml.PauliZ(1))

In [3]:
def my_parameter_shift_grad(params, shift):
    """Your homemade parameter-shift rule function.
    
    NOTE: you cannot use qml.grad within this function

    Args:
        params (list(float)): The parameters for gates in the circuit

    Returns:
        gradient (numpy.array): The gradient of the circuit with respect to the given parameters.
    """
    gradient = np.zeros_like(params)
    s = shift
    
    for i in range(len(params)):
        #first we need to make two copies of the parameter list 
        param_first = np.zeros_like(params)
        param_second = np.zeros_like(params)
        #now we have to change their values in order to make the (Theta+s) (Theta-s)
        for k in  range(len(params)):
            if k == i:
                param_first[k] = (params[k] + s)
                param_second[k] = (params[k] - s)
            else:
                param_first[k] = params[k] 
                param_second[k] = params[k]
        #now lets calculate the numerator
        numerator = (circuit(param_first) - circuit(param_second))
        #calculating the denominator
        denominator = 2 * np.sin(s)
        #final gradient 
        gradient[i] = numerator / denominator
        
    return np.round_(gradient, decimals=5).tolist()

In [4]:
# These functions are responsible for testing the solution.

def run(test_case_input: str) -> str:
    params, shift = json.loads(test_case_input)
    gradient = my_parameter_shift_grad(params, shift)
    return str(gradient)

def check(solution_output: str, expected_output: str) -> None:
    solution_output = json.loads(solution_output)
    expected_output = json.loads(expected_output)
    assert np.allclose(
        solution_output, expected_output, rtol=1e-4
    ), "Your gradient isn't quite right!"

In [5]:
test_cases = [['[[0.75, 1.0], 1.23]', '[-0.68164, -0.84147]']]

In [6]:
for i, (input_, expected_output) in enumerate(test_cases):
    print(f"Running test case {i} with input '{input_}'...")

    try:
        output = run(input_)

    except Exception as exc:
        print(f"Runtime Error. {exc}")

    else:
        if message := check(output, expected_output):
            print(f"Wrong Answer. Have: '{output}'. Want: '{expected_output}'.")

        else:
            print("Correct!")

Running test case 0 with input '[[0.75, 1.0], 1.23]'...
Correct!
